In [52]:
import pandas as pd
from tqdm import tqdm
import snscrape.modules.twitter as sntwitter
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import datetime as dt
from textblob import TextBlob

nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/chloe_ycl/nltk_data...


True

In [38]:
query = "$TSLA"
tweets_list = []
until = dt.datetime.strftime(dt.date.today(), '%Y-%m-%d')
since = dt.datetime.strftime(dt.datetime.strptime(until, '%Y-%m-%d') - dt.timedelta(days=5), '%Y-%m-%d')
n_tweets = 2000

# count -1 mean get all the rows
for i, tweet in tqdm(enumerate(sntwitter.TwitterSearchScraper(f'{query} until:{until} since:{since} filter:links -filter:replies -count:-1').get_items()), total=n_tweets):
    tweets_list.append([tweet.date, tweet.id, tweet.rawContent, tweet.user, tweet.cashtags])
    if i > n_tweets:
        break
#Creating a dataframe from the tweets list above 
df = pd.DataFrame(tweets_list, columns=['date', 'id', 'content', 'user','cashtag'])
df.tail(10)

2001it [01:59, 16.75it/s]                          


,date,id,content,user,cashtag
1992,2023-03-03 19:06:31+00:00,1631732803573526528,Minimize your investment with us join us https...,https://twitter.com/Keanu5004,"[DKNG, SPY, QQQ, IWM, SLV, MGM, ROKU, AAPL, AM..."
1993,2023-03-03 19:06:02+00:00,1631732682278473729,Enhance your trading experience with us click ...,https://twitter.com/Keanu5004,"[APT, JAKK, DYNT, SPV, LMND, GNUS, GRAF, BIOL,..."
1994,2023-03-03 19:05:47+00:00,1631732621393936384,Most Profitable trading community \nJoin our d...,https://twitter.com/bishnuvardhan,"[SPY, TSLA, NIO, EYES, QQQ, UVXY, ES, FB, BA]"
1995,2023-03-03 19:05:24+00:00,1631732524719243277,$SONY #SONY #PlayStation and LeBron James team...,https://twitter.com/Wallstreetrehab,"[SONY, ATVI, NTDOY, RKUNY, EA, MSFT, AAPL, MP,..."
1996,2023-03-03 19:05:24+00:00,1631732523590995968,Tesla heeft in februari 74.402 in China gemaak...,https://twitter.com/BeursforumBE,[TSLA]
1997,2023-03-03 19:05:11+00:00,1631732470545633280,This is how Tesla will fight FUD and advertise...,https://twitter.com/DoctorJack16,[TSLA]
1998,2023-03-03 19:05:11+00:00,1631732469484638209,MAX PAIN for 3/3/23 12:04 PM\nTicker: $SPY Max...,https://twitter.com/CeoPurchases,"[SPY, QQQ, IWM, TSLA, SPY]"
1999,2023-03-03 19:05:10+00:00,1631732465395007499,🟢 Trending right now\n\n$AI $28 +32.7%\n$DMTK ...,https://twitter.com/tradesense_push,"[AI, DMTK, NVAX, TSLA, PRAX, VERU, CDNA, ZS]"
2000,2023-03-03 19:04:58+00:00,1631732414832836608,$TSLA Everyone is Buying the Dip Right Now.Hoa...,https://twitter.com/VelmaCh59575276,[TSLA]
2001,2023-03-03 19:04:55+00:00,1631732401511489536,Mexican president Andrés Manuel Lopez Obrador ...,https://twitter.com/OZLithiumCorp,"[OZLI, NLC, RCK, TSLA, ML]"


In [39]:
df.head(5)

,date,id,content,user,cashtag
0,2023-03-04 23:58:20+00:00,1632168629545365507,If you were clueless about Master Plan 3 … the...,https://twitter.com/itechnosmith,[TSLA]
1,2023-03-04 23:57:12+00:00,1632168343972249600,特斯拉 ( $TSLA ) 股價上漲 69%，成為 2023 年表現最佳的標普;P 500 ...,https://twitter.com/deloris888,[TSLA]
2,2023-03-04 23:56:17+00:00,1632168114564497408,Economy 101 lesson from Professor Dr. Elon Mus...,https://twitter.com/teslionaire,"[TSLA, Tslaq]"
3,2023-03-04 23:55:23+00:00,1632167889062232065,1st V4 supercharger coming soon? @TeslaChargin...,https://twitter.com/WaiWai29225868,[tsla]
4,2023-03-04 23:48:12+00:00,1632166079559774210,Donald Musk...\n\n$TSLA $TSLAQ,https://twitter.com/passthebeano,"[TSLA, TSLAQ]"


In [43]:
# Create a function to clean the tweets
def clean_text(text):
    text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
    text = re.sub('#', '', text) # Removing '#' hash tag
    text = re.sub('\n', '', text) # Removing \n
    text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
    return text

#applying this function to Text column of our dataframe
df["content"] = df["content"].apply(clean_text)

df.head(5)

,date,id,content,user,cashtag
0,2023-03-04 23:58:20+00:00,1632168629545365507,If you were clueless about Master Plan 3 … the...,https://twitter.com/itechnosmith,[TSLA]
1,2023-03-04 23:57:12+00:00,1632168343972249600,特斯拉 ( $TSLA ) 股價上漲 69%，成為 2023 年表現最佳的標普;P 500 ...,https://twitter.com/deloris888,[TSLA]
2,2023-03-04 23:56:17+00:00,1632168114564497408,Economy 101 lesson from Professor Dr. Elon Mus...,https://twitter.com/teslionaire,"[TSLA, Tslaq]"
3,2023-03-04 23:55:23+00:00,1632167889062232065,1st V4 supercharger coming soon? $tsla,https://twitter.com/WaiWai29225868,[tsla]
4,2023-03-04 23:48:12+00:00,1632166079559774210,Donald Musk...$TSLA $TSLAQ,https://twitter.com/passthebeano,"[TSLA, TSLAQ]"


In [49]:
def get_tweet_sentiment(tweet):
    '''
    Utility function to classify sentiment of passed tweet
    using SentimentIntensityAnalyzer's sentiment method
    '''
    # create SentimentIntensityAnalyzer object of passed tweet text
    # return a dict of results
    SIA_obj = SentimentIntensityAnalyzer()
    analysis = SIA_obj.polarity_scores(tweet)

    # decide sentiment as positive, negative and neutral
    if analysis['neg'] > analysis['pos']:
        return 'negative'
    elif analysis['neg'] < analysis['pos']:
        return 'positive'
    else:
        return 'neutral'

In [50]:
def get_tweet_sentiment_score(tweet):
    '''
    Utility function to classify sentiment of passed tweet
    using SentimentIntensityAnalyzer's sentiment method
    '''
    # create SentimentIntensityAnalyzer object of passed tweet text
    # return a dict of results
    SIA_obj = SentimentIntensityAnalyzer()
    analysis = SIA_obj.polarity_scores(tweet)

    # decide sentiment as positive, negative and neutral
    if analysis['neg'] > analysis['pos']:
        return analysis['neg']
    elif analysis['neg'] < analysis['pos']:
        return analysis['pos']
    else:
        return analysis['neu']

In [53]:
df['sentiment'] = df['content'].apply(get_tweet_sentiment)

df['score'] = df['content'].apply(get_tweet_sentiment_score)

df.head(10)

,date,id,content,user,cashtag,sentiment,score
0,2023-03-04 23:58:20+00:00,1632168629545365507,If you were clueless about Master Plan 3 … the...,https://twitter.com/itechnosmith,[TSLA],negative,0.200
1,2023-03-04 23:57:12+00:00,1632168343972249600,特斯拉 ( $TSLA ) 股價上漲 69%，成為 2023 年表現最佳的標普;P 500 ...,https://twitter.com/deloris888,[TSLA],positive,0.091
2,2023-03-04 23:56:17+00:00,1632168114564497408,Economy 101 lesson from Professor Dr. Elon Mus...,https://twitter.com/teslionaire,"[TSLA, Tslaq]",negative,0.121
3,2023-03-04 23:55:23+00:00,1632167889062232065,1st V4 supercharger coming soon? $tsla,https://twitter.com/WaiWai29225868,[tsla],neutral,1.000
4,2023-03-04 23:48:12+00:00,1632166079559774210,Donald Musk...$TSLA $TSLAQ,https://twitter.com/passthebeano,"[TSLA, TSLAQ]",neutral,1.000
5,2023-03-04 23:47:28+00:00,1632165895559864320,We finally got the new ToS login screen! $SPY ...,https://twitter.com/5ABTRDR,"[SPY, SPX, TSLA]",neutral,1.000
6,2023-03-04 23:46:31+00:00,1632165657784758272,Should I take my clothes off or keep it on? $T...,https://twitter.com/Teslaconomics,[TSLA],neutral,1.000
7,2023-03-04 23:40:59+00:00,1632164264919068672,$HUBC reminds me of $PGY only more potential! ...,https://twitter.com/AirGoodman24,"[HUBC, PGY, LUNR, AMC, APE, TRKA, COSM, BBIG, ...",positive,0.062
8,2023-03-04 23:40:46+00:00,1632164208069718017,These are the 3 aftermarket add-ons I purchase...,https://twitter.com/FSDMaestro,[TSLA],positive,0.176
9,2023-03-04 23:38:49+00:00,1632163719714336768,"This is going nowhere fast...""Tesla said it “d...",https://twitter.com/passthebeano,"[TSLA, TSLAQ]",positive,0.216


In [61]:
df['date']= df['date'].apply(lambda x: x.strftime('%d-%m-%Y'))
df.head(5)

/tmp/ipykernel_23126/3783080406.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date']= df['date'].apply(lambda x: x.strftime('%d-%m-%Y'))


,date,rawcontent,id,user,cashtags,sentiment,polarity_score
0,03-03-2023,Anyone want know how to blow your account in 5...,1631742040286576640,"{'username': 'smalltrdacc_23', 'id': 159299615...","[TSLA, NVDA, QQQ, SPX, SPY, MFST, NFLX, SPY, M...",positive,0.40
1,02-03-2023,"PressRelease$KEGS - Signs MOU with ERApeutics,...",1631336581788311558,"{'username': 'InvestorshubT', 'id': 1576965168...","[KEGS, TWTR, META, ACB, MFST]",positive,0.25
2,02-03-2023,Those businesses will not be its Tesla’s AW...,1631320188657319941,"{'username': 'rmcadory', 'id': 14283770, 'disp...","[TSLA, AAPL, MFST]",positive,0.25
3,01-03-2023,Paid the price for breaking my rules. Market t...,1631015139662270464,"{'username': 'smalltrdacc_23', 'id': 159299615...","[TSLA, NVDA, QQQ, SPX, SPY, MFST, NFLX, SPY, M...",neutral,0.00
4,01-03-2023,$MFSTWindowsにも対話AI検索「Bing」導入拡大：NIKKEI👉2/28にはWi...,1630757147956105217,"{'username': '2525mementomori', 'id': 11553269...",[MFST],neutral,0.00


In [62]:
df[['day', 'month', 'year']] = df.date.str.split("-", expand=True)

df.head(5)

/tmp/ipykernel_23126/2923524131.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['day', 'month', 'year']] = df.date.str.split("-", expand=True)
/tmp/ipykernel_23126/2923524131.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['day', 'month', 'year']] = df.date.str.split("-", expand=True)
/tmp/ipykernel_23126/2923524131.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

,date,rawcontent,id,user,cashtags,sentiment,polarity_score,day,month,year
0,03-03-2023,Anyone want know how to blow your account in 5...,1631742040286576640,"{'username': 'smalltrdacc_23', 'id': 159299615...","[TSLA, NVDA, QQQ, SPX, SPY, MFST, NFLX, SPY, M...",positive,0.40,03,03,2023
1,02-03-2023,"PressRelease$KEGS - Signs MOU with ERApeutics,...",1631336581788311558,"{'username': 'InvestorshubT', 'id': 1576965168...","[KEGS, TWTR, META, ACB, MFST]",positive,0.25,02,03,2023
2,02-03-2023,Those businesses will not be its Tesla’s AW...,1631320188657319941,"{'username': 'rmcadory', 'id': 14283770, 'disp...","[TSLA, AAPL, MFST]",positive,0.25,02,03,2023
3,01-03-2023,Paid the price for breaking my rules. Market t...,1631015139662270464,"{'username': 'smalltrdacc_23', 'id': 159299615...","[TSLA, NVDA, QQQ, SPX, SPY, MFST, NFLX, SPY, M...",neutral,0.00,01,03,2023
4,01-03-2023,$MFSTWindowsにも対話AI検索「Bing」導入拡大：NIKKEI👉2/28にはWi...,1630757147956105217,"{'username': '2525mementomori', 'id': 11553269...",[MFST],neutral,0.00,01,03,2023
